## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [1]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [2]:
import pandas as pd
import numpy as np

# Step 1. Read csv file
df = pd.read_csv('/drive/MyDrive/Colab_Notebooks/Data_Analyst_Course_HP/Module_6/Data_M6/cookie_cats.csv')

In [3]:

# Step 2. Calculating the mean of retention_7 by game version
mean_retention = df.groupby('version')['retention_7'].mean()
print(mean_retention)


version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64


**Hypotheses:**

H0: The retention rate after 7 days does not depend on the version of the game.

H1: Version gate_30 has a higher retention rate after 7 days compared to version gate_40.

**Preliminary conclusion:**

The average retention_7 value for version gate_30 (0.190201) is higher than for version gate_40 (0.182000).
Based on this data, version gate_30 may be more promising for increasing player retention after 7 days.

2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [10]:
import numpy as np
from statsmodels.stats.weightstats import ztest
from scipy.stats import norm

# Step 1. Divide data into groups
gate_30 = df[df['version'] == 'gate_30']['retention_7'].to_numpy()
gate_40 = df[df['version'] == 'gate_40']['retention_7'].to_numpy()

# Step 2. Perform a two-sample z-test
z_stat, p_value = ztest(gate_30, gate_40)
print(f"z statistic: {z_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Step 3. Calculate confidence intervals (95%)
alpha = 0.05
z_critical = norm.ppf(1 - alpha / 2)

mean_30, std_30 = np.mean(gate_30), np.std(gate_30, ddof=1)
mean_40, std_40 = np.mean(gate_40), np.std(gate_40, ddof=1)

ci_30 = (mean_30 - z_critical * (std_30 / np.sqrt(len(gate_30))),
         mean_30 + z_critical * (std_30 / np.sqrt(len(gate_30))))

ci_40 = (mean_40 - z_critical * (std_40 / np.sqrt(len(gate_40))),
         mean_40 + z_critical * (std_40 / np.sqrt(len(gate_40))))

print(f"Confidence interval 95% for the gate_30 group: {ci_30}")
print(f"Confidence interval 95% for the gate_40 group: {ci_40}")

z statistic: 3.1645
p-value: 0.0016
Confidence interval 95% for the gate_30 group: (0.18656307582527862, 0.19383960873847975)
Confidence interval 95% for the gate_40 group: (0.17845426175887072, 0.18554582617447582)


**Interpretation:**

z statistic and p-value:

Since the p-value (0.0016) < α (0.05), we reject the null hypothesis (H0).
This means that retention_7 is statistically significantly different between versions of the game.
The gate_30 version shows better retention after 7 days than the gate_40 version.

Confidence intervals:

For gate_30: [0.1866, 0.1938]
For gate_40: [0.1785, 0.1855].

The intervals do not overlap, confirming a statistically significant difference between the two groups.

**Conclusion:**

The gate_30 version of the game has a significantly better retention rate after 7 days compared to the gate_40 version. It is recommended to focus on maintaining or further developing this version to improve player retention.

3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


In [9]:
from scipy.stats import chi2_contingency

# Step 1. Create a 2x2 contiguity table
contingency_table = pd.crosstab(df['version'], df['retention_7'])

# Step 2. Conducting a chi-square test
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# Displaying the results
print(f"Chi-square statistic: {chi2_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Step 3. Conclusion based on p-value
alpha = 0.05
if p_value <= alpha:
    print("\nSince the p-value <= 0.05, we reject the null hypothesis.")
    print("There is a statistically significant relationship between the game version and retention_7.")
else:
    print("\nSince the p-value is > 0.05, we cannot reject the null hypothesis.")
    print("There is insufficient evidence of a relationship between game version and retention_7.")

# Step 4. Displaying the adjacency table to view the distribution of data
print("\nTable of conjugacy:")
print(contingency_table)

Chi-square statistic: 9.9591
p-value: 0.0016

Since the p-value <= 0.05, we reject the null hypothesis.
There is a statistically significant relationship between the game version and retention_7.

Table of conjugacy:
retention_7  False  True 
version                  
gate_30      36198   8502
gate_40      37210   8279


**Analysis:**

***Interpretation of p-value:***

**p-value** (0.0016) < α (0.05)

This means that we reject the null hypothesis
There is a statistically significant relationship between game version and retention_7

**Comparison with the previous z-test:**

Z-test: p-value = 0.0016
Chi-square test: p-value = 0.0016

1. Both tests give identical p-values, which strengthens the reliability of the results
2. Both tests indicate a statistically significant difference between the versions

Overall conclusion:
1. There is a statistically significant relationship between the game version and the probability of a user returning on day 7.
2. Version gate_30 shows better retention results.
3. The results of both statistical tests (z-test and chi-square) are consistent with each other.
4. It is recommended to use gate_30 version for better user retention.